In [4]:
import pandas as pd
import numpy as np 
import gspread
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv 
hv.extension('bokeh')


In [14]:
df = pd.read_csv(r'C:\Users\tsheb\OneDrive\Desktop\k\money_out_statement2.csv', sep = ';')
print(df)


        Nr     Account Posting Date  transaction date  \
0        1  1844042632   2025/02/01  2025/02/01 00:53   
1        2  1844042632   2025/02/01  2025/02/01 00:53   
2        3  1844042632   2025/02/01  2025/02/01 01:15   
3        4  1844042632   2025/02/01  2025/01/30 16:36   
4        5  1844042632   2025/02/01  2025/02/01 11:18   
...    ...         ...          ...               ...   
1186  1187  1844042632   2026/02/11  2026/02/11 11:20   
1187  1188  1844042632   2026/02/11  2026/02/11 16:33   
1188  1189  1844042632   2026/02/11  2026/02/11 16:33   
1189  1190  1844042632   2026/02/12  2026/02/12 01:06   
1190  1191  1844042632   2026/02/12  2026/02/09 10:17   

                                            description  \
0                 Banking App Prepaid Purchase: Vodacom   
1                           Prepaid Mobile Purchase Fee   
2                            Live Better Interest Sweep   
3                     Intercape East London (Card 1291)   
4                 Ba

In [15]:
# clean data
df = df[['transaction date','original description', 'category', 'money out updated']] # keeping the desired columns only
df.head()
df['transaction date'] = pd.to_datetime(df['transaction date'])
df['Month'] = df['transaction date'].dt.month
df['Year'] = df['transaction date'].dt.year
pd.options.display.max_rows = 999
df.head(200)



,transaction date,original description,category,money out updated,Month,Year
0,2025-02-01 00:53:00,Vodacom,Cellphone,-10.00,2,2025
1,2025-02-01 00:53:00,Prepaid Mobile Purchase Fee,Fees,-0.50,2,2025
2,2025-02-01 01:15:00,LB Interest sweep | JAN 2025,Transfer,-0.49,2,2025
3,2025-01-30 16:36:00,Intercape ESL Station East London ZA,Public Transport,-31.50,1,2025
4,2025-02-01 11:18:00,Vodacom,Cellphone,-12.00,2,2025
5,2025-02-01 11:18:00,Prepaid Mobile Purchase Fee,Fees,-0.50,2,2025
6,2025-02-01 13:41:00,Round-up Transfer,Transfer,-1.00,2,2025
7,2025-02-01 20:52:00,Vodacom,Cellphone,-12.00,2,2025
8,2025-02-01 20:52:00,Prepaid Mobile Purchase Fee,Fees,-0.50,2,2025
9,2025-02-02 03:26:00,Round-up Transfer,Transfer,-0.50,2,2025


In [16]:
# Summary  banner of last months income, recuring expenses non recurring expenses and savings
# get the latest month and year

latest_month = df['Month'].max()
latest_year = df['Year'].max()

# filter r=the data frame to include only trabsactions from the latest month
last_month_expenses = df[(df['Month'] == latest_month) & (df['Year'] == latest_year)]

In [17]:
df['transation date '] = pd.to_datetime(df['transaction date'])

latest_period = df['transaction date'].dt.to_period('M').max()

last_month_expenses = df[
    df['transaction date'].dt.to_period('M') == latest_period
]


In [18]:
last_month_expenses = last_month_expenses.groupby('category')['money out updated'].sum().reset_index()
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].astype('str')
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].str.replace('-','')
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].astype('float')

last_month_expenses = last_month_expenses[last_month_expenses['category'].str.contains('excluded|unassoigned')== False]
last_month_expenses = last_month_expenses.sort_values(by='money out updated', ascending = False) # sort bakues
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].round().astype(int) # round vallues
last_month_expenses 
#



,category,money out updated
1,Cash Withdrawal,700
2,Cellphone,74
0,Betting/Lottery,65
3,Fees,31
5,Transfer,2
4,Other Income,0


In [19]:
# Group and sum by category
last_month_expenses = last_month_expenses.groupby('category')['money out updated'].sum().reset_index()

# Make values positive (cleaner way)
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].abs()

# Sort largest to smallest
last_month_expenses = last_month_expenses.sort_values(by='money out updated', ascending=False)

# Round values
last_month_expenses['money out updated'] = last_month_expenses['money out updated'].round().astype(int)

last_month_expenses


,category,money out updated
1,Cash Withdrawal,700
2,Cellphone,74
0,Betting/Lottery,65
3,Fees,31
5,Transfer,2
4,Other Income,0


In [20]:
print(last_month_expenses.shape)
last_month_expenses_tot = last_month_expenses['money out updated'].sum()
last_month_expenses_tot


(6, 2)


np.int64(872)

In [21]:
# define the widgets in the dashboard
def calculate_difference(event):
    income = float(income_widget.value)
    recurring_expenses = float(recurring_expenses_widget.value)
    monthly_expenses = float(monthly_expenses_widget.value)
    difference = income - recurring_expenses - monthly_expenses
    difference_widget.value = str(difference)

income_widget = pn.widgets.TextInput(name = 'Income' , value ='0')
recurring_expenses_widget = pn.widgets.TextInput( name = 'Recurring expenses' , value ='0')
monthly_expenses_widget = pn.widgets.TextInput(name = 'Non-recurring expenses' , value = str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput( name = 'last months savings' , value = '0')

income_widget.param.watch(calculate_difference, 'value')  
recurring_expenses_widget.param.watch(calculate_difference,'value')  
monthly_expenses_widget.param.watch(calculate_difference,'value')  
pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()


Launching server at http://localhost:51823


In [22]:

income_widget = pn.widgets.TextInput(name = 'Income' , value ='0')
recurring_expenses_widget = pn.widgets.TextInput( name = 'Recurring expenses' , value ='0')
monthly_expenses_widget = pn.widgets.TextInput(name = 'Non-recurring expenses' , value = str(last_month_expenses_tot))
difference_widget = pn.widgets.TextInput( name = 'last months savings' , value = '0')

income_widget.param.watch(calculate_difference, 'value')  
recurring_expenses_widget.param.watch(calculate_difference,'value')  
monthly_expenses_widget.param.watch(calculate_difference,'value')  
pn.Row(income_widget, recurring_expenses_widget, monthly_expenses_widget, difference_widget).show()


Launching server at http://localhost:51824


In [23]:
#last onths expenses by category
last_month_expenses_chart = last_month_expenses.hvplot.bar(
    x = 'category', 
    y = 'money out updated' , 
    height = 300,
    width = 900,
    title = ' Last Month Expenses',
    ylim = (0,900))
last_month_expenses_chart

:Bars   [category]   (money out updated)

In [24]:
# create a monthly expenses trend bar chart
df['transaction date'] =pd.to_datetime(df['transaction date'])
df['Month-Year'] = df['transaction date'].dt.to_period('M') 
monthly_expenses_trend_by_cat = df.groupby(['Month-Year' , 'category'])['money out updated'].sum().reset_index()
monthly_expenses_trend_by_cat['money out updated'] = monthly_expenses_trend_by_cat['money out updated'].astype('str') 
monthly_expenses_trend_by_cat['money out updated'] = monthly_expenses_trend_by_cat['money out updated'].str.replace('-', '')
monthly_expenses_trend_by_cat['money out updated'] = monthly_expenses_trend_by_cat['money out updated'].astype('float') 
monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat[monthly_expenses_trend_by_cat ['category'].str.contains('xcluded|unassoigned')== False]

monthly_expenses_trend_by_cat = monthly_expenses_trend_by_cat.sort_values(by='money out updated', ascending = False)
monthly_expenses_trend_by_cat['money out updated']= monthly_expenses_trend_by_cat['money out updated'].round().astype(int) 
monthly_expenses_trend_by_cat['Month-Year'] = monthly_expenses_trend_by_cat['Month-Year'].astype(str) 
monthly_expenses_trend_by_cat


,Month-Year,category,money out updated
51,2025-05,Software/Games,4680
73,2025-07,Sport & Hobbies,2829
26,2025-03,Sport & Hobbies,2232
106,2025-10,Digital Payments,1970
70,2025-07,Groceries,1769
58,2025-06,Groceries,1631
55,2025-06,Cash Withdrawal,1500
75,2025-07,Transfer,1425
5,2025-02,Digital Payments,1358
8,2025-02,Groceries,1353


In [ ]:
# Define panel widget 
#categories = df['Category'].dropna().unique().tolist()
categories = df['category'].dropna().unique().tolist()
categories


category_widget = pn.widgets.Select(
    name='category',
    options=sorted(categories))

category_widget



BokehModel(combine_events=True, render_bundle={'docs_json': {'717ecd77-6ad6-49d1-bbd2-c67ef0426abb': {'version…